In [2]:
import sys, traceback
import os
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from datetime import date, timedelta, datetime

In [3]:
def scaled(image):  # for LANDSAT8 hyperspectral
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    emis = image.select('ST_EMIS').multiply(0.0001)  # estimated emissivity
    emsd = image.select('ST_EMSD').multiply(0.0001)  # uncertainty in emissivity estimation
    trad = image.select('ST_TRAD').multiply(0.001)  # thermal band converted to radiance
    qa = image.select('ST_QA').multiply(0.01)  # uncertainty in land surface temperature
    
    image =  image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
    image = image.addBands(emis, None, True).addBands(emsd, None, True)
    image = image.addBands(trad, None, True).addBands(qa, None, True)
    return image

# Thermal Band Converted to Radiance

In [112]:
min_, max_ = 0, 11
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))
left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_TRAD']), {'min': min_, 'max': max_}, 'TIR')
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': min_, 'max': max_}, label="Thermal Band Radiance (W/(m^2*sr*um)/ DN)")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [113]:
# B10 TOA
# min_, max_ = 200, 310
# Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
# sample_toa = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_025039_20210807")
# Map.addLayer(sample_toa.select(['B10']), {'min': min_, 'max': max_,}, 'Landsat8')
# Map.add_colorbar({'min': min_, 'max': max_}, label="Thermal Band B10 TOA")
# Map

In [114]:
# min_, max_ = 280, 350
# Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
# sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))

# left_layer = geemap.ee_tile_layer(sample_toa.select(['B10']), {'min': min_, 'max': max_,}, 'B10 TOA')
# right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_B10']), {'min': min_, 'max': max_,}, 'TIR')
# Map = geemap.Map(center=(29.7662, -95.2410), zoom=14)
# Map.split_map(left_layer=left_layer, right_layer=right_layer)
# Map.add_colorbar({'min': min_, 'max': max_,}, label="Brightness Temperature (K)")
# Map

# ASTER GED Estimated Emissivity

In [4]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))
min_, max_ = 0.85, 1.0
left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_EMIS']), {'min': min_, 'max': max_}, 'TIR')
Map = geemap.Map(center=(29.7662, -95.2410), zoom=14)
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': min_, 'max': max_}, label="Estimated Emissivity")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

# Band 10 Surface Temperature

In [116]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))

left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_B10']), {'min': 300, 'max': 350,}, 'TIR')
Map = geemap.Map(center=(29.7662, -95.2410), zoom=14)
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': 320, 'max': 370,}, label="Surface Temperature (K)")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

# Emissivity Standard Deviation

In [117]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))
min_, max_ = 0, .15
left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_EMSD']), {'min': min_, 'max': max_}, 'TIR')
Map = geemap.Map(center=(29.7662, -95.2410), zoom=14)
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': min_, 'max': max_}, label="Emissivity Standard Deviation")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

# Uncertainty of the Surface Temperature

In [119]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))
min_, max_ = 0, 10
left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_QA']), {'min': min_, 'max': max_}, 'TIR')
Map = geemap.Map(center=(29.7662, -95.2410), zoom=14)
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': min_, 'max': max_}, label="Uncertainty in Surface Temperature (K)")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Cloud: up to 10 Kelvin</br>
Cloud-free: 4-6 Kelvin

In [125]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
sample_tile = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807")
min_, max_ = 0, 1
left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile.select(['ST_ATRAN']).multiply(0.0001), {'min': min_, 'max': max_}, 'TIR')
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': min_, 'max': max_}, label="Atmospheric Transmittance")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

# LANDSAT8 & LANDSAT 9 (8-day difference)

In [173]:
min_, max_ = 0, 11
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
landsat8 = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20220215"))
landsat9 = scaled(ee.Image("LANDSAT/LC09/C02/T1_L2/LC09_025039_20220207"))
# landsat9 = scaled(ee.Image("LANDSAT/LC09/C02/T1_L2/LC09_025039_20211205"))

left_layer = geemap.ee_tile_layer(landsat8.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(landsat9.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat9')
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': min_, 'max': max_}, label="Thermal Band Radiance (W/(m^2*sr*um)/ DN)")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…